In [14]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Input, MaxPooling2D,  Dropout, Rescaling
from keras.applications import MobileNetV2
from keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model


def dataset_split(data, train_split=0.85, val_split=0.15, shuffle=True, shuffle_size=5700):
    if shuffle:
        data = data.shuffle(shuffle_size)
    data_size = len(data)
    train_size = int(data_size * train_split)
    train_set = data.take(train_size)
    val_size = int(data_size * val_split)
    remaining_data = data.skip(train_size)
    val_set = remaining_data.take(val_size)
    
    return train_set, val_set


training_data = image_dataset_from_directory( "Brain_tumor/Training", pad_to_aspect_ratio=True )
testing_data = image_dataset_from_directory( "Brain_tumor/Testing", pad_to_aspect_ratio=True )
train_set, val_set = dataset_split( data= training_data )

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.


In [15]:
num_classes = 4

model = Sequential()

# Input layer
model.add(Input(shape=(256, 256, 3)))

# First Convolutional layer with ReLU
model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu'))

# Average Pooling layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# Second Convolutional layer with ReLU
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Average Pooling layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

# Flattening layer
model.add(Flatten())

# First Dense layer with ReLU
model.add(Dense(units=120, activation='relu'))  # Updated to 100 neurons

# Second Dense layer with ReLU
model.add(Dense(units=84, activation='relu'))  # Updated to 75 neurons

# Output layer with Softmax
model.add(Dense(units=10, activation='softmax'))  # Output layer neurons set to number of classes

# Compile the model
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to verify the architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 128, 128, 6)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 62, 62, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 120)            │     7,380,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,394,486 (28.21 MB)

 Trainable params: 7,394,486 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history1 = model.fit( train_set, validation_data=val_set, epochs=20 )

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 27s 139ms/step - accuracy: 0.2536 - loss: 3.9735 - val_accuracy: 0.2320 - val_loss: 1.4424
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 24s 143ms/step - accuracy: 0.2617 - loss: 1.4158 - val_accuracy: 0.2452 - val_loss: 1.3930
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 128ms/step - accuracy: 0.2545 - loss: 1.4044 - val_accuracy: 0.2320 - val_loss: 1.4051
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.2727 - loss: 1.3956 - val_accuracy: 0.2620 - val_loss: 1.4088
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 124ms/step - accuracy: 0.2550 - loss: 1.4003 - val_accuracy: 0.2945 - val_loss: 1.3981
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - accuracy: 0.2605 - loss: 1.3980 - val_accuracy: 0.2536 - val_loss: 1.4028
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.2617 - loss: 1.3969 - val_accuracy: 0.2849 - val_loss: 1.3890
Epoch 8/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 126ms/step - accuracy: 0.2542 - loss: 1

In [17]:
training_accuracy = history1.history['accuracy']
validation_accuracy = history1.history['val_accuracy']
training_accuracy = [ round( num, 3 ) for num in training_accuracy ]
validation_accuracy = [ round( num, 3 ) for num in validation_accuracy ]

print( training_accuracy )
print( validation_accuracy )

[0.261, 0.264, 0.263, 0.266, 0.253, 0.262, 0.27, 0.256, 0.253, 0.258, 0.262, 0.267, 0.26, 0.266, 0.259, 0.27, 0.263, 0.261, 0.256, 0.268]
[0.232, 0.245, 0.232, 0.262, 0.294, 0.254, 0.285, 0.244, 0.27, 0.262, 0.272, 0.258, 0.288, 0.263, 0.269, 0.287, 0.276, 0.278, 0.291, 0.275]


In [18]:
num_classes = 4

model2 = Sequential()

# Insert Rescaling layer
model2.add(Rescaling(1./255, input_shape=(256, 256, 3)))

# First Convolutional layer with ReLU
model2.add(Conv2D(filters=6, kernel_size=(5, 5), padding='same', activation='relu'))

# Replace with MaxPooling
model2.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Second Convolutional layer with ReLU
model2.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))

# Replace with MaxPooling
model2.add(MaxPooling2D(pool_size=(2, 2), strides=2))

# Flatten before passing to the Dense layers
model2.add(Flatten())

# First Dense layer with ReLU
model2.add(Dense(units=120, activation='relu'))

# Insert Dropout layer here with a 0.2 dropout rate
model2.add(Dropout(0.2))

# Second Dense layer with ReLU
model2.add(Dense(units=84, activation='relu'))

# Insert Dropout layer here with a 0.2 dropout rate
model2.add(Dropout(0.2))

# Output layer with Softmax
# Make sure the number of units equals the number of classes
model2.add(Dense(units=10, activation='softmax'))

# Compile the model
model2.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to verify the architecture
model2.summary()


C:\Users\zafee\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:18: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 6)    │           456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 124, 124, 16)   │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 61504)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 120)            │     7,380,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 84)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,394,486 (28.21 MB)

 Trainable params: 7,394,486 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history2 = model2.fit( train_set, validation_data=val_set, epochs=20 )

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 123ms/step - accuracy: 0.9792 - loss: 0.0637 - val_accuracy: 0.9976 - val_loss: 0.0234
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 22s 125ms/step - accuracy: 0.9792 - loss: 0.0607 - val_accuracy: 0.9940 - val_loss: 0.0205
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.9791 - loss: 0.0584 - val_accuracy: 0.9808 - val_loss: 0.0517
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 119ms/step - accuracy: 0.9848 - loss: 0.0535 - val_accuracy: 0.9988 - val_loss: 0.0065
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 120ms/step - accuracy: 0.9862 - loss: 0.0402 - val_accuracy: 0.9988 - val_loss: 0.0077
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 119ms/step - accuracy: 0.9830 - loss: 0.0454 - val_accuracy: 1.0000 - val_loss: 0.0095
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 119ms/step - accuracy: 0.9854 - loss: 0.0450 - val_accuracy: 0.9988 - val_loss: 0.0103
Epoch 8/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 21s 119ms/step - accuracy: 0.9890 - loss: 0

In [37]:
training_accuracy2 = history2.history['accuracy']
validation_accuracy2 = history2.history['val_accuracy']
training_accuracy2 = [ round( num, 3 ) for num in training_accuracy2 ]
validation_accuracy2 = [ round( num, 3 ) for num in validation_accuracy2 ]

print( training_accuracy2 )
print( validation_accuracy2 )

[0.476, 0.635, 0.726, 0.772, 0.812, 0.837, 0.854, 0.866, 0.883, 0.898, 0.91, 0.916, 0.933, 0.94, 0.95, 0.953, 0.956, 0.967, 0.965, 0.971]
[0.659, 0.745, 0.799, 0.862, 0.84, 0.838, 0.865, 0.925, 0.892, 0.941, 0.962, 0.974, 0.921, 0.98, 0.962, 0.987, 0.99, 0.99, 0.994, 0.969]


In [38]:
test_accuracy2 = model2.evaluate( testing_data)
print( test_accuracy2 )

41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.9207 - loss: 0.1960
[0.2236362248659134, 0.9145690202713013]


In [6]:
# Load MobileNetV2 without the top classification layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Add new layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  # Adjust dropout rate as needed
predictions = Dense(4, activation='softmax')(x)

# This is the model we will train
model3 = Model(inputs=base_model.input, outputs=predictions)
model3.summary( )

C:\Users\zafee\AppData\Local\Temp\ipykernel_3448\34579375.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 128, 128,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 128, 128,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 128, 128,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 128, 128,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 128, 128,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 128, 128,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 128, 128,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 128, 128,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 129, 129,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 64, 64,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 64, 64,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 64, 64,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 64, 64,    │      2,304 │ block_1_depthwis

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 2,228,996 (8.50 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [8]:
print( len( base_model.layers))

154


In [9]:
# To make the last 12 layers trainable:
for layer in base_model.layers[:-12]:
    layer.trainable = False

# Now, when you check the model summary again:
model3.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 128, 128,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 128, 128,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 128, 128,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 128, 128,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 128, 128,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 128, 128,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 128, 128,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 128, 128,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 128, 128,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 129, 129,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 64, 64,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 64, 64,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 64, 64,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 64, 64,    │      2,304 │ block_1_depthwis

 Total params: 2,263,108 (8.63 MB)

 Trainable params: 891,204 (3.40 MB)

 Non-trainable params: 1,371,904 (5.23 MB)

In [11]:
# Compile the model (as shown in the previous example)
model3.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Start the timer
start_time = time.time()

# Fit the model
history3 = model3.fit(train_set,
                    epochs=20,  # replace with the number of epochs you desire
                    validation_data=val_set)

# End the timer
end_time = time.time()

# Calculate and print the training time
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 122s 746ms/step - accuracy: 0.7245 - loss: 0.6979 - val_accuracy: 0.3223 - val_loss: 2.2110
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 116s 744ms/step - accuracy: 0.8876 - loss: 0.2970 - val_accuracy: 0.3810 - val_loss: 2.2134
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 108s 697ms/step - accuracy: 0.9070 - loss: 0.2533 - val_accuracy: 0.4892 - val_loss: 1.5137
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 105s 671ms/step - accuracy: 0.9298 - loss: 0.1949 - val_accuracy: 0.6022 - val_loss: 1.0785
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 103s 660ms/step - accuracy: 0.9436 - loss: 0.1781 - val_accuracy: 0.5793 - val_loss: 1.0963
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 107s 685ms/step - accuracy: 0.9518 - loss: 0.1446 - val_accuracy: 0.7800 - val_loss: 0.5662
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 102s 656ms/step - accuracy: 0.9569 - loss: 0.1277 - val_accuracy: 0.8162 - val_loss: 0.4656
Epoch 8/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 103s 662ms/step - accuracy: 0.9638 -

In [22]:
test_accuracy1 = model.evaluate( testing_data)
test_accuracy2 = model2.evaluate( testing_data)
test_accuracy3 = model3.evaluate( testing_data)


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.2456 - loss: 1.3911
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9563 - loss: 0.1130
41/41 ━━━━━━━━━━━━━━━━━━━━ 21s 511ms/step - accuracy: 0.9434 - loss: 0.1759


In [23]:
print( test_accuracy1 )
print( test_accuracy2 )
print( test_accuracy3 )

[1.391426682472229, 0.22883294522762299]
[0.13517984747886658, 0.9549961686134338]
[0.17241258919239044, 0.942791759967804]
